In [68]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import warnings
from multiprocessing import Pool
import tensorflow as tf
from tensorflow.python.ops import nn

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Lambda, Dropout
from tensorflow.keras import Input, Model

py_file_location = "..."
os.path.abspath(os.path.join(os.path.dirname(py_file_location), os.path.pardir))





tf.device('GPU:1') 

## Loading the data

In [69]:
Nbr_train = int(1E6)

Nbr_test = int(2E5) 

#outfile = 'Dataset/' # choose your path 

In [70]:
Nbr_test_bruteforce = 10000


project_sub_path = "Imperfect CSI Data/CF_DNN"
  
# Parent Directory path
parent_dir = ""
### Train ###
#dataset_train_Anne
dataset_train = np.load(os.path.join(parent_dir,project_sub_path,'dataset_train_GF.npz'))

h_11_tr = dataset_train['h_PP']
h_12_tr = dataset_train['h_PS']
h_1R_tr = dataset_train['h_PR']
h_21_tr = dataset_train['h_SP']
h_22_tr = dataset_train['h_SS']
h_2R_tr = dataset_train['h_SR']
h_R1_tr = dataset_train['h_RP']
h_R2_tr = dataset_train['h_RS']


#x_train_input = np.stack([h_11_tr, h_12_tr, h_1R_tr, h_21_tr, h_22_tr, h_2R_tr, h_R1_tr, h_R2_tr], axis=1)
x_train = np.stack([h_R1_tr, h_11_tr, h_2R_tr, h_1R_tr, h_22_tr, h_R2_tr, h_21_tr, h_12_tr], axis=1)





### VAL ###
  
# Parent Directory path
#dataset_val_Anne

dataset_test = np.load(os.path.join(parent_dir,project_sub_path,'dataset_val_GF.npz'))

h_11_test = dataset_test['h_PP'][:Nbr_test_bruteforce]
h_12_test = dataset_test['h_PS'][:Nbr_test_bruteforce]
h_1R_test = dataset_test['h_PR'][:Nbr_test_bruteforce]
h_21_test = dataset_test['h_SP'][:Nbr_test_bruteforce]
h_22_test = dataset_test['h_SS'][:Nbr_test_bruteforce]
h_2R_test = dataset_test['h_SR'][:Nbr_test_bruteforce]
h_R1_test = dataset_test['h_RP'][:Nbr_test_bruteforce]
h_R2_test = dataset_test['h_RS'][:Nbr_test_bruteforce]

x_test = np.stack([h_R1_test, h_11_test, h_2R_test, h_1R_test, h_22_test, h_R2_test, h_21_test, h_12_test], axis=1)




# CF loss function

In [71]:
def loss_CF(Lambda=10**0.5, v_tau=0.25):
    
    def CF_loss(G, y_out):
        
        ''' compute loss for CF Relaying'''
        
        Tau = tf.constant(v_tau, dtype=tf.float32) # ==> Tau 

        W = tf.constant(Lambda, dtype=tf.float32)  # ==> lambda 

        G = tf.cast(G, dtype='float32')
        
        y_out = tf.cast(y_out, dtype='float32')

        # index retrieval

        Grp_indx, Gpp_indx, Gsr_indx, Gpr_indx, Gss_indx, Grs_indx, Gsp_indx, Gps_indx  = [0], [1], [2], [3], [4], [5], [6], [7]
        
        Pr_indx, Ps_indx  = [0], [1]

        # tensors retrieval
        
        Grp, Gpp, Gsr, Gpr, Gss, Grs, Gsp, Gps, Pr, Ps = tf.gather(G, Grp_indx, axis=1), tf.gather(G, Gpp_indx, axis=1), tf.gather(G, Gsr_indx, axis=1), tf.gather(G, Gpr_indx, axis=1), tf.gather(G, Gss_indx, axis=1), tf.gather(G, Grs_indx, axis=1), tf.gather(G, Gsp_indx, axis=1), tf.gather(G, Gps_indx, axis=1), tf.gather(y_out, Pr_indx, axis=1), tf.gather(y_out, Ps_indx, axis=1)

        #  Primary power Creation

        Pp = tf.multiply(tf.ones(tf.shape(Pr), dtype=tf.dtypes.float32),10.0)

        # NS_Tilde :NS = gPS*PP +1

        NS_Tilde = tf.add(tf.multiply(Gps,Pp),tf.constant(1, dtype=tf.float32))

        #NR_Tilde : gPR*PP +1

        NR_Tilde = tf.add(tf.multiply(Gpr,Pp),tf.constant(1, dtype=tf.float32))

        # Rho_Z : sqrt(Gpr*Gps)*Pp/sqrt(NR_tilde*NS_tilde)

        Rho_Z = tf.divide(tf.multiply(tf.sqrt(tf.multiply(Gpr,Gps)),Pp),tf.sqrt(tf.multiply(NR_Tilde,NS_Tilde)))

        #K1 : Gsr*NS_Tilde+Gss*NR_Tilde-2*Rho_Z*sqrt(Gsr*Gss*NR_Tilde*NS_Tilde)

        E1 = tf.add(tf.multiply(Gsr,NS_Tilde),tf.multiply(Gss,NR_Tilde))

        E2 = tf.sqrt(tf.multiply(tf.multiply(tf.multiply(Gsr,Gss),NR_Tilde),NS_Tilde))

        K1 = tf.math.subtract(E1,tf.multiply(tf.multiply(tf.constant(2, dtype=tf.float32),Rho_Z),E2))

        #K2 : (1-Rho_Z**2)*NR_Tilde*NS_Tilde
        
        K2 = tf.multiply(tf.math.subtract(tf.constant(1, dtype=tf.float32),tf.pow(Rho_Z,tf.constant(2, dtype=tf.float32))),tf.multiply(NR_Tilde,NS_Tilde))

        # function A' ==> A'(Gpp) : ((Gpp*Pp)/((1+(Gpp*Pp))**(1-tau)-1))-1 ==> (Gpp*Pp)/(R1) 

        R1 = tf.add(tf.constant(1, dtype=tf.float32),tf.multiply(Gpp,Pp))
        R1 = tf.pow(R1, tf.math.subtract(tf.constant(1, dtype=tf.float32),Tau))
        R1 = tf.math.subtract(R1,tf.constant(1, dtype=tf.float32))

        A_ = tf.subtract(tf.divide(tf.multiply(Gpp,Pp),R1),tf.constant(1, dtype=tf.float32))

        # QoS : 

        # PR**2 and PS**2 because custom_sigmoid(x):  
        # returns : Output of sigmoid function range between 0 and sqrt(10)

        Qos = tf.add(tf.multiply(Gsp,tf.pow(Ps,2)),tf.multiply(Grp,tf.pow(Pr,2)))

        Qos = tf.subtract(Qos, A_)

        Qos = tf.multiply(W,tf.keras.activations.relu(Qos)) 

        # SNR : 
        num = tf.add(tf.multiply(tf.multiply(tf.multiply(K1,Grs),\
                                             tf.pow(Ps,2)),tf.pow(Pr,2)),\
                     tf.add(tf.multiply(tf.multiply(Gss,tf.pow(Ps,2)),tf.multiply(K1,tf.pow(Ps,2))),\
                     tf.multiply(tf.multiply(Gss,tf.pow(Ps,2)),K2)))
        
        d_num = tf.add(tf.multiply(tf.multiply(K2,Grs), tf.pow(Pr,2)),\
                       tf.add(tf.multiply(NS_Tilde,tf.multiply(K1,tf.pow(Ps,2))),\
                       tf.multiply(NS_Tilde, K2)))
        
        SNR_opt = tf.divide(num, d_num)

        # R_S

        Rs_opt =  tf.multiply(tf.constant(0.5, dtype=tf.float32),log2(tf.add(tf.constant(1,dtype=tf.float32),SNR_opt)))

        #-n_SNR+n_Qos
        res = tf.reduce_mean(-Rs_opt+Qos) 


        return res
    return CF_loss



# DNN Model 

In [72]:
def get_model_CF(X_train, f_loss, f_metrics, f_activation1, f_activation2, LR) :
    """

      Structure of DL model for DF.

      Parameters:
         X_train: Channel gain array.
         f_loss: Loss function.
         f_metrics: List of metrics.
         f_activation1: First activation function for the output.
         f_activation2: Second activation function for the output.
         LR : Learning rate.
         
      Returns:
        DL model for CF.
    """
    opt = tf.keras.optimizers.Adam(learning_rate = LR)
    inputs = Input(shape=(X_train.shape[1]))
    x = Dense(128, activation='relu')(inputs)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    output1 = Dense(1, activation=f_activation1)(x)
    output2 = Dense(1, activation=f_activation2)(x)
    merged = tf.keras.layers.Concatenate()([output1, output2])
    model = Model(inputs=inputs, outputs=[merged])
    model.compile(loss=f_loss, optimizer=opt, metrics=f_metrics)
    model.summary()
    return model



# Performance metrics

In [73]:
#------------ tensorflow functions for DNN ------------# 
def custom_sigmoid(x):
  """
    Modified sigmoid function used for handling predicted powers.

    Parameters:
      x: tensor.
    Returns:
      Output of sigmoid function range between 0 and sqrt(10)
  """
  output = tf.multiply(tf.sqrt(tf.constant(10,dtype=tf.float32)),nn.sigmoid(x))
  # Cache the logits to use for crossentropy loss.
  output._keras_logits = x  # pylint: disable=protected-access
  return output


def log2(x):
  numerator = tf.math.log(x)
  denominator = tf.math.log(tf.constant(2, dtype=tf.float32))
  return numerator / denominator
def Delta_DNN(G, y_out):
 
    """
      Metrics used on DL model for Primary achievable rate degradation.
      
      Parameters:
        G: Channel gain tensor.
        y_out: Predicted parameter.
      Returns:
        Primary achievable rate degradation for each samples 
    """
    G = tf.cast(G, dtype='float32')
    y_out = tf.cast(y_out, dtype='float32')
    
    # index retrieval


    Grp_indx, Gpp_indx, Gsp_indx = [0], [1], [6]
    Pr_indx, Ps_indx  = [0], [1]

    # tensors retrieval
    Grp, Gpp, Gsp, Pr, Ps = tf.gather(G, Grp_indx, axis=1), tf.gather(G, Gpp_indx, axis=1), tf.gather(G, Gsp_indx, axis=1),  tf.gather(y_out, Pr_indx, axis=1), tf.gather(y_out, Ps_indx, axis=1)
    
    
    #  Pp Creation
    Pp = tf.multiply(tf.ones(tf.shape(Pr), dtype=tf.dtypes.float32),10)
    
    # Rp : C((Gpp*Pp)/(Grp*Pr**2+Gsp*Ps**2+2*(np.sqrt(Gsp*Grp)*Alpha*Ps*Pr)+1) ==> C(H1/H2), where H1 = Gpp*Pp and H2 = R1+R2+1
    H1 = tf.multiply(Gpp, Pp)

    R1 =  tf.add(tf.multiply(Grp, tf.pow(Pr, 2)),tf.multiply(Gsp,tf.pow(Ps, 2)))

    H2 = tf.add(R1, tf.constant(1,dtype=tf.float32))
    
    SNR_P = tf.divide(H1, H2)
    
    Rp =  tf.multiply(tf.constant(0.5, dtype=tf.float32),log2(tf.add(tf.constant(1,dtype=tf.float32),SNR_P)))

    # Rp_ : C(Gpp*Pp)
    SNR_P_ = tf.multiply(Gpp, Pp)
    Rp_ = tf.multiply(tf.constant(0.5,dtype=tf.float32),log2(tf.add(tf.constant(1,dtype=tf.float32),SNR_P_)))
    # 1 - ratio(Rp, Rp_)
    RRP = tf.subtract(tf.constant(1,dtype=tf.float32), tf.divide(Rp, Rp_))
    
    return RRP



def Opportunistic_Achievable_Rate(v_tau):
    def opportunistic_rate(G, y_out):
        """
        Metrics used on DL model for throughput calculation.
        This function will get those parameters as input
        G: Channel gain tensor.
        y_out: Predicted parameter.

        Parameters:
        Lambda : Penalty for QoS
        Tau : degradation factor for the primary network
        Returns:
        Throughput mean 
        """

        Tau = tf.constant(v_tau, dtype=tf.float32) # ==> Tau 

        G = tf.cast(G, dtype='float32')
        y_out = tf.cast(y_out, dtype='float32')

        # index retrieval
    


        Grp_indx, Gpp_indx, Gsr_indx, Gpr_indx, Gss_indx, Grs_indx, Gsp_indx, Gps_indx = [0], [1], [2], [3], [4], [5], [6], [7]
        Pr_indx, Ps_indx  = [0], [1]

        # tensors retrieval
        Grp, Gpp, Gsr, Gpr, Gss, Grs, Gsp, Gps, Pr, Ps = tf.gather(G, Grp_indx, axis=1), tf.gather(G, Gpp_indx, axis=1), tf.gather(G, Gsr_indx, axis=1), tf.gather(G, Gpr_indx, axis=1), tf.gather(G, Gss_indx, axis=1), tf.gather(G, Grs_indx, axis=1), tf.gather(G, Gsp_indx, axis=1), tf.gather(G, Gps_indx, axis=1), tf.gather(y_out, Pr_indx, axis=1), tf.gather(y_out, Ps_indx, axis=1)
        Pp = tf.multiply(tf.ones(tf.shape(Pr), dtype=tf.dtypes.float32),10)


        NS_Tilde = tf.add(tf.multiply(Gps,Pp),tf.constant(1, dtype=tf.float32))

        #NR_Tilde : gPR*PP +1

        NR_Tilde = tf.add(tf.multiply(Gpr,Pp),tf.constant(1, dtype=tf.float32))

        # Rho_Z : sqrt(Gpr*Gps)*Pp/sqrt(NR_tilde*NS_tilde)

        Rho_Z = tf.divide(tf.multiply(tf.sqrt(tf.multiply(Gpr,Gps)),Pp),tf.sqrt(tf.multiply(NR_Tilde,NS_Tilde)))


        #K1 : Gsr*NS_Tilde+Gss*NR_Tilde-2*Rho_Z*sqrt(Gsr*Gss*NR_Tilde*NS_Tilde)

        E1 = tf.add(tf.multiply(Gsr,NS_Tilde),tf.multiply(Gss,NR_Tilde))

        E2 = tf.sqrt(tf.multiply(tf.multiply(tf.multiply(Gsr,Gss),NR_Tilde),NS_Tilde))

        K1 = tf.math.subtract(E1,tf.multiply(tf.multiply(tf.constant(2, dtype=tf.float32),Rho_Z),E2))

        #K2 : (1-Rho_Z**2)*NR_Tilde*NS_Tilde

        K2 = tf.multiply(tf.math.subtract(tf.constant(1, dtype=tf.float32),tf.pow(Rho_Z,tf.constant(2, dtype=tf.float32))),tf.multiply(NR_Tilde,NS_Tilde))

        num = tf.add(tf.multiply(tf.multiply(tf.multiply(K1,Grs),tf.pow(Ps,2)),tf.pow(Pr,2)),tf.add(tf.multiply(tf.multiply(Gss,tf.pow(Ps,2)),tf.multiply(K1,tf.pow(Ps,2))),tf.multiply(tf.multiply(Gss,tf.pow(Ps,2)),K2)))
                
        d_num = tf.add(tf.multiply(tf.multiply(K2,Grs), tf.pow(Pr,2)),tf.add(tf.multiply(NS_Tilde,tf.multiply(K1,tf.pow(Ps,2))),tf.multiply(NS_Tilde,K2)))
        
        SNR_opt = tf.divide(num, d_num)

        #C = 1/2*log2(1+SNR_opt)
        # debit calculation
        Rs = tf.multiply(tf.constant(0.5,dtype=tf.float32),log2(tf.add(tf.constant(1,dtype=tf.float32),SNR_opt)))
        return Rs
    return opportunistic_rate


def outage_percentage(v_tau): 
    def outage_percentage(G, y_out): 
        """
          metrics used on DL model for testing Primary achievable rate degradation percentage .

          Parameters:
            G: Channel gain tensor.
            y_out: Predicted parameter.
          Returns:
            percentage of Primary achievable rate degradation 
        """
        Tau = tf.constant(v_tau, dtype=tf.float32) # ==> Tau 

        G = tf.cast(G, dtype='float32')

        y_out = tf.cast(y_out, dtype='float32')

        # index retrieval


        Grp_indx, Gpp_indx, Gsp_indx = [0], [1], [6]
        Pr_indx, Ps_indx  = [0], [1]

        # tensors retrieval
        Grp, Gpp, Gsp, Pr, Ps = tf.gather(G, Grp_indx, axis=1), tf.gather(G, Gpp_indx, axis=1), tf.gather(G, Gsp_indx, axis=1),  tf.gather(y_out, Pr_indx, axis=1), tf.gather(y_out, Ps_indx, axis=1)


        #  Pp Creation
        Pp = tf.multiply(tf.ones(tf.shape(Pr), dtype=tf.dtypes.float32),10)

        # Rp : C((Gpp*Pp)/(Grp*Pr**2+Gsp*Ps**2+2*(np.sqrt(Gsp*Grp)*Alpha*Ps*Pr)+1) ==> C(H1/H2), where H1 = Gpp*Pp and H2 = R1+R2+1
        H1 = tf.multiply(Gpp, Pp)

        R1 =  tf.add(tf.multiply(Grp, tf.pow(Pr, 2)),tf.multiply(Gsp,tf.pow(Ps, 2)))

        H2 = tf.add(R1, tf.constant(1,dtype=tf.float32))

        SNR_P = tf.divide(H1, H2)

        Rp =  tf.multiply(tf.constant(0.5, dtype=tf.float32),log2(tf.add(tf.constant(1,dtype=tf.float32),SNR_P)))

        # Rp_ : C(Gpp*Pp)
        SNR_P_ = tf.multiply(Gpp, Pp)
        Rp_ = tf.multiply(tf.constant(0.5,dtype=tf.float32),log2(tf.add(tf.constant(1,dtype=tf.float32),SNR_P_)))
        # 1 - ratio(Rp, Rp_)
        RRP = tf.subtract(tf.constant(1,dtype=tf.float32), tf.divide(Rp, Rp_))

        # 1 - ratio(Rp, Rp_)
        ARD = tf.subtract(tf.constant(1,dtype=tf.float32), tf.divide(Rp, Rp_))

        #ARD > tau  
        mask_PDD = tf.greater(ARD, Tau)# boolean array 

        return mask_PDD #*100 ?? 

    return outage_percentage


def QoS_Violation(v_tau, QoS_thresh = -5): 
    def V_Qos(G, y_out): 
        """
        metrics used on DL model for testing QoS viloation .

        Parameters:
            G: Channel gain tensor.
            y_out: Predicted parameter.
        Returns:
            Number of violated QoS 
        """
        Tau = tf.constant(v_tau, dtype=tf.float32) # ==> Tau 

        W = tf.constant(Lambda, dtype=tf.float32)  # ==> lambda 

        G = tf.cast(G, dtype='float32')

        y_out = tf.cast(y_out, dtype='float32')

        # index retrieval

        Grp_indx, Gpp_indx, Gsr_indx, Gpr_indx, Gss_indx, Grs_indx, Gsp_indx, Gps_indx  = [0], [1], [2], [3], [4], [5], [6], [7]

        Pr_indx, Ps_indx  = [0], [1]

        # tensors retrieval

        Grp, Gpp, Gsr, Gpr, Gss, Grs, Gsp, Gps, Pr, Ps = tf.gather(G, Grp_indx, axis=1), tf.gather(G, Gpp_indx, axis=1), tf.gather(G, Gsr_indx, axis=1), tf.gather(G, Gpr_indx, axis=1), tf.gather(G, Gss_indx, axis=1), tf.gather(G, Grs_indx, axis=1), tf.gather(G, Gsp_indx, axis=1), tf.gather(G, Gps_indx, axis=1), tf.gather(y_out, Pr_indx, axis=1), tf.gather(y_out, Ps_indx, axis=1)

        #  Primary power Creation

        Pp = tf.multiply(tf.ones(tf.shape(Pr), dtype=tf.dtypes.float32),10.0)

        # function A' ==> A'(Gpp) : ((Gpp*Pp)/((1+(Gpp*Pp))**(1-tau)-1))-1 ==> (Gpp*Pp)/(R1) 

        R1 = tf.add(tf.constant(1, dtype=tf.float32),tf.multiply(Gpp,Pp))
        R1 = tf.pow(R1, tf.math.subtract(tf.constant(1, dtype=tf.float32),Tau))
        R1 = tf.math.subtract(R1,tf.constant(1, dtype=tf.float32))

        A_ = tf.subtract(tf.divide(tf.multiply(Gpp,Pp),R1),tf.constant(1, dtype=tf.float32))

        # QoS : 

        # PR**2 and PS**2 because custom_sigmoid(x):  
        # returns : Output of sigmoid function range between 0 and sqrt(10)

        Qos = tf.add(tf.multiply(Gsp,tf.pow(Ps,2)),tf.multiply(Grp,tf.pow(Pr,2)))

        Qos = tf.subtract(Qos, A_)

        Qos = tf.multiply(W,tf.keras.activations.relu(Qos)) 

        Qos = tf.subtract(Qos, A_)

        #n_Qos = tf.divide(Qos, A_) # Normalization ?????
        #Qos > 10**-5  
        mask_pr = tf.greater(n_Qos,tf.math.pow(tf.constant(10, dtype=tf.float32),QoS_thresh))# boolean array 

        return mask_pr
    return V_Qos


In [65]:
Nbr_train = int(1E6)

 
tau = 0.25

VS = 0.2 # validation_split

Epochs = 10 # Epochs number

BS = 4096 # batch_size

LD = 10**0.5

LR = 10**-4

metrics = [Delta_DNN, Opportunistic_Achievable_Rate(tau),outage_percentage(tau)] #, QoS_mean_DF, QoS_median_DF



#Nbr_train = int(1E6)

model = get_model_CF(x_train, loss_CF(LD,tau), metrics, custom_sigmoid, custom_sigmoid, LR) #lr_v
history = model.fit(x_train, np.power(x_train,2), epochs=Epochs, batch_size=BS, validation_split = VS)






Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 8)]          0                                            
__________________________________________________________________________________________________
dense_60 (Dense)                (None, 128)          1152        input_11[0][0]                   
__________________________________________________________________________________________________
dense_61 (Dense)                (None, 256)          33024       dense_60[0][0]                   
__________________________________________________________________________________________________
dense_62 (Dense)                (None, 256)          65792       dense_61[0][0]                   
___________________________________________________________________________________________

# Part 2 

In [66]:
N2, NR, N1= 1.0, 1.0, 1.0
P2_max, PR_max, P1 = 10.0, 10.0, 10.0
tau = 0.25

def calculate_A(H11): 
    return ((((H11**2)*P1)/(((1+(((H11**2)*P1)))**(1-tau))-1))-1)    

def C(x): 
    return (1/2*np.log2(1+x))


def Delta(HR1, H11, H2R, H1R, H22, HR2, H21, H12):
   
    #A = (((H11**2*P1)/(((1+((H11**2*P1)/(N1)))**(1-tau))-1))-N1)    
    A = calculate_A(H11)
    N2_tilde = ((H12**2)*P1+1)
    NR_tilde = ((H1R**2)*P1+1)
    P_Z = ((H12*H1R*P1)/(np.sqrt(N2_tilde*NR_tilde)))
    K1 = ((H2R**2)*N2_tilde+(H22**2)*NR_tilde-2*H2R*H22*P_Z*np.sqrt(N2_tilde*NR_tilde))
    K2 = ((1-P_Z**2)*NR_tilde*N2_tilde)
    
    C1 = K1*(HR1**2)*((H22**2)*(HR1**2)-(HR2**2)*(H21**2))
    C2 = K1*(HR2**2)*(H21**2)*A-2*(HR1**2)*(H22**2)*K1*A-(HR1**2)*(H22**2)*(H21**2)*K2
    C3 = (H22**2)*K1*(A**2)+(H22**2)*(H21**2)*K2*A
    C4 = K2*(HR2**2)*(H21**2)**2-N2_tilde*K1*(H21**2)*(HR1**2)
    C5 = N2_tilde*(H21**2)*(K1*A+K2*(H21**2))
    D = C1**2*C5**2-C1*C4*(C2*C5-C3*C4)
    
    return C1,C2,C3,C4,C5,D

def f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr, P2):
    # f en fonction de PR et P2
    A = calculate_A(H11)     

    N2_tilde = ((H12**2)*P1+1)
    NR_tilde = ((H1R**2)*P1+1)
    P_Z = ((H12*H1R*P1)/(np.sqrt(N2_tilde*NR_tilde)))
    K1 = ((H2R**2)*N2_tilde+(H22**2)*NR_tilde-2*H2R*H22*P_Z*np.sqrt(N2_tilde*NR_tilde))
    K2 = ((1-P_Z**2)*NR_tilde*N2_tilde)
    #res = (K1*HR2**2*P2*Pr+H22**2*P2*K1*P2+H22**2*P2*K2)/(K2*HR2**2*Pr+N2_tilde*K1*P2+N2_tilde*K2)
    res = (K1*(HR2**2)*P2*Pr+(H22**2)*P2*(K1*P2+K2))/(K2*(HR2**2)*Pr+N2_tilde*K1*P2+N2_tilde*K2)

    
    return res

def f(HR1, H11, H2R, H1R, H22, HR2, H21, H12, x):
    'fonction depend seulment de Pr (x apres changement de variable) '
    # f en fonction de PR 
    A = calculate_A(H11)
    N2_tilde = ((H12**2)*P1+1)
    NR_tilde = ((H1R**2)*P1+1)
    P_Z = ((H12*H1R*P1)/(np.sqrt(N2_tilde*NR_tilde)))
    K1 = ((H2R**2)*N2_tilde+(H22**2)*NR_tilde-2*H2R*H22*P_Z*np.sqrt(N2_tilde*NR_tilde))
    K2 = ((1-P_Z**2)*NR_tilde*N2_tilde)


    C1 = K1*(HR1**2)*((H22**2)*(HR1**2)-(HR2**2)*(H21**2))
    C2 = K1*(HR2**2)*(H21**2)*A-2*(HR1**2)*(H22**2)*K1*A-(HR1**2)*(H22**2)*(H21**2)*K2
    C3 = (H22**2)*K1*(A**2)+(H22**2)*(H21**2)*K2*A
    C4 = K2*(HR2**2)*(H21**2)**2-N2_tilde*K1*(H21**2)*(HR1**2)
    C5 = N2_tilde*(H21**2)*(K1*A+K2*(H21**2))

    f = (C1*x**2+C2*x+C3)/(C4*x+C5)
    
    return f
def CF_V4(HR1, H11, H2R, H1R, H22, HR2, H21, H12):
    '''
    Becarful of channels ID order (check the Analytic solution file channels ID order)!!!!!!
    HR2, H12, H1R, HR1, H22, H2R, H21, H11
    '''
        
    A = calculate_A(H11)
    #x_min, x_max = f_domain(HR2, H12, H1R, HR1, H22, H2R, H21, H11)
    if ( (A/(HR1**2)) < PR_max and (A/(H21**2)) < P2_max ) :  # H1

        x_min, x_max = 0.0, A/(HR1**2)        

    elif ( (A/(HR1**2)) < PR_max and (A/(H21**2)) > P2_max ):# H2

        x_min, x_max =  (A-(H21**2)*P2_max)/(HR1**2), A/(HR1**2)

    elif ( (A/(HR1**2)) > PR_max and (A/(H21**2)) < P2_max ):# H3

        x_min, x_max =  0.0 , PR_max
        
    elif ( (A/(HR1**2)) > PR_max and (A/(H21**2)) > P2_max ) and P2_max*(H21**2) + PR_max*(HR1**2) > A: #H4

        x_min, x_max =  (A-(H21**2)*P2_max)/(HR1**2), PR_max
        
    C1, C2, C3, C4, C5, delta = Delta(HR1, H11, H2R, H1R, H22, HR2, H21, H12)

    try :
        if C1*C4>0:
            #print('C1*C4>0')
            
            if delta > 0 :
                x_1 = (-(C1*C5)-np.sqrt(delta))/(C1*C4) 
                x_2 = (-(C1*C5)+np.sqrt(delta))/(C1*C4) 
                if (x_1 <= x_min <= x_max <= x_2): 
                    Pr_opt = x_min 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt,P2_opt)
                elif (x_1<=x_min<=x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_min)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)): 
                    Pr_opt = x_min 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt,P2_opt)
                      
                elif x_1 >= x_max :
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
               
                elif x_2 <= x_max :
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)

                elif  (x_1 <= x_min <= x_2 <= x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_min)):
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                    
                elif (x_min <= x_1 <= x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_1)): 
                    Pr_opt = x_max 
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)

                elif (x_min <= x_1 <= x_max <=x_2) :
                    Pr_opt = x_1
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                elif (x_min<=x_1<=x_2<=x_max and f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_1)>f(HR2, H12, H1R, HR1, H22, H2R, H21, H11, x_max)):
                    Pr_opt = x_1
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


            elif delta == 0 :
                
                x_0 = -C5/C4
                Pr_opt = x_max
                P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                
            else :
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                    


        elif C1*C4 == 0.0 : 
            #print('C1*C4 == 0')
            x_ = -(C2*C5-C3*C4)/(2*C1*C5)

            if (C1*C5)>0.0 :
                
                if (C2*C5-C3*C4)>0.0 :

                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)

                else : 

                    if x_ < x_min : 

                        Pr_opt = x_max
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)

                    elif (x_min <= x_ <= x_max) : 

                        if f(HR1, H11, H2R, H1R, H22, HR2, H21, H12, x_min)>f(HR1, H11, H2R, H1R, H22, HR2, H21, H12, x_max) :
                            
                            Pr_opt = x_min
                            P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                            SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


                        else : 
                            
                            Pr_opt = x_max
                            P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                            SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


                    elif x_ > x_max :

                        Pr_opt = x_min
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


            elif (C1*C5) == 0.0 : 
                #print('C1*C5')
                if (C2*C5-C3*C4) > 0.0 :
                    
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)

                else :
                    
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


            else :
                if (C2*C5-C3*C4)>0.0 :
                    #print((C2*C5-C3*C4))
                    if x_ < x_min :
                        
                        Pr_opt = x_min
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


                    elif (x_min <= x_ <= x_max):
                        
                        Pr_opt = x_
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)

                    elif x_ > x_max :
                        
                        Pr_opt = x_max  #     x_    
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                
                else : # (C2*C5-C3*C4) < 0 ...
                        Pr_opt = x_min        
                        P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                        SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


        else :
            #print('C1*C4<0')

            if delta > 0.0 :
                #print('delta>0')
                x_1 = (-(C1*C5)-np.sqrt(delta))/(C1*C4) 
                x_2 = (-(C1*C5)+np.sqrt(delta))/(C1*C4)
                
                if x_1 <= x_min  : 
                    
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                
                elif (x_min<=x_2<=x_max and f(HR1, H11, H2R, H1R, H22, HR2, H21, H12, x_min)>f(HR1, H11, H2R, H1R, H22, HR2, H21, H12,x_max)): 
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


                elif  x_max <= x_2:
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                elif  (x_min<=x_2<=x_max<=x_1 and f(HR1, H11, H2R, H1R, H22, HR2, H21, H12, x_max)>f(HR1, H11, H2R, H1R, H22, HR2, H21, H12,x_min)):
                    Pr_opt = x_min
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)


                elif (x_min <= x_1 <= x_max) :
                    
                    Pr_opt = x_1
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                elif (x_min <= x_2 <= x_max <= x_1 and f(HR1, H11, H2R, H1R, H22, HR2, H21, H12, x_max)>f(HR1, H11, H2R, H1R, H22, HR2, H21, H12,x_min)) :
                    
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                elif (x_2 <= x_min <= x_max <= x_1) :
                    
                    Pr_opt = x_max
                    P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                    SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
               
                
            elif delta == 0 :
                #print('delta=0')


                x_0 = -C5/C4
                Pr_opt = x_min
                P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                
            else :
            
                Pr_opt = x_min
                P2_opt = (A-(HR1**2)*Pr_opt)/(H21**2)
                SNR = f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt)
                

     

    except UnboundLocalError:
        
        Pr_opt, P2_opt, SNR = PR_max, P2_max ,f_obj(HR1, H11, H2R, H1R, H22, HR2, H21, H12, PR_max, P2_max)
        #print('[H5]-[H6]')
    
    
    return HR1, H11, H2R, H1R, H22, HR2, H21, H12, Pr_opt, P2_opt, SNR
            
    
    

In [67]:
def noise_to_channels(X, primary_ID, secondary_ID, SNRs_db = [-10, -5, 0, 5, 10, 15, 20]):
    '''
    Parameters : 
       
        test_set :  test set containing the H channels
    
        col : list of index for the specific column to add noise
    
    Returns:
    
        channel gain ndarray container of noisy channels 

    '''   
   
    static_var_X = np.sqrt(np.var(X[:, primary_ID], axis=0))

    var_X = np.var(X[:, secondary_ID], axis=0, keepdims=True)
    
    noisy_gains = [] # list to store all the noisy H matrices with different level of noise variance
    
    for SNR_db in SNRs_db:
        
        SNR = np.power(10,SNR_db/10)
        noises = np.sqrt(var_X/SNR)*np.random.normal(0.0, 1.0, (X.shape[0], len(secondary_ID)))
    
        X_noised = X.copy()
        X_noised[:, secondary_ID] = X_noised[:, secondary_ID] + noises
        X_noised[:, primary_ID] = static_var_X
        noisy_gains.append(X_noised)

    return SNRs_db, np.asarray(noisy_gains, dtype="float64")
#-----------------------------------------------------------------------------------------#

def BF_A_squeeze(x):
    return CF_V4(x[0], x[1], x[2], x[3], x[4], x[5], x[6], x[7])

def generate_benchmark(H_matrix): 
    '''
    bruteforce for H without noise
    '''
    with Pool() as p:
        BF_res =  p.map(BF_A_squeeze, np.power(H_matrix, 2))

    return np.squeeze(np.asarray(BF_res, dtype="float64"))

#-----------------------------------------------------------------------------------------#

def AS_for_noisy_channels(BH_matrix):
    
    '''
    Compute bruteforce method for channel gain ndarray composed of noisy channels
    '''

    BF_res = [] # list containing channels and bruteforce results (Alpha,Pr,Ps) for each noisy matrix (0, 10^-1.5, 10^-1....) 
    
    
    for i in range(BH_matrix.shape[0]) :
        X = BH_matrix[i,:,:]
        
        temp_BF_res = generate_benchmark(X)
        
        BF_res.append(temp_BF_res)

        
    return np.asarray(BF_res, dtype="float64")


#-----------------------------------------------------------------------------------------#





In [ ]:
# Get P and S channels index form dataset (h_R1, h_11, h_2R, h_1R, h_22, h_R2, h_21, h_12)

Primary_ID = [0, 1, 6] 
Secondary_ID = [3, 7]


noise_levels, X_train_noised = noise_to_channels(x_train, Primary_ID, Secondary_ID)
X_train_noised_path = 'Imperfect CSI Data/CF_DNN/X_train_noised_GF_CDIT'
np.savez(X_train_noised_path, X_train_noised)




In [19]:
#X_train_noised_GF = 'Dataset_VF/X_train_noised_anne'
X_train_noised_path = 'Imperfect CSI Data/CF_DNN/X_train_noised_GF_CDIT'

X_train_noised = np.load(X_train_noised_path+".npz")
X_train_noised.files
X_train_noised = X_train_noised['arr_0']



In [46]:
# Get Noisy H-MATRIX with different noise variance

noise_levels, NH_MATRIX = noise_to_channels(x_test, Primary_ID, Secondary_ID)

# Compute bruteforce for NH-MATRIX

AS_NG_MATRIX = AS_for_noisy_channels(NH_MATRIX) 

# bruteforce for test_set without noise

AS_G_Benchmark = generate_benchmark(x_test) 

#----------------------------------Save data----------------------------------------------#

outfile_NH_MATRIX = 'Imperfect CSI Data/CF_DNN/NH_MATRIX_GF_CDIT'
np.savez(outfile_NH_MATRIX, NH_MATRIX)


outfile_BF_NG_MATRIX = 'Imperfect CSI Data/CF_DNN/BF_NG_MATRIX_GF_CDIT'
np.savez(outfile_BF_NG_MATRIX, AS_NG_MATRIX)


outfile_BF_G_Benchmark = 'Imperfect CSI Data/CF_DNN/BF_G_Benchmark_GF_CDIT'
np.savez(outfile_BF_G_Benchmark, AS_G_Benchmark)

/tmp/ipykernel_189571/1096670552.py:6: RuntimeWarning: divide by zero encountered in double_scalars
  return ((((H11**2)*P1)/(((1+(((H11**2)*P1)))**(1-tau))-1))-1)
/tmp/ipykernel_189571/1096670552.py:23: RuntimeWarning: invalid value encountered in double_scalars
  C2 = K1*(HR2**2)*(H21**2)*A-2*(HR1**2)*(H22**2)*K1*A-(HR1**2)*(H22**2)*(H21**2)*K2


UnboundLocalError: local variable 'Pr_opt' referenced before assignment

In [ ]:
# First, load ... 
# useful for DNN prediction because DNN takes H as input 

outfile_NH_MATRIX = 'Imperfect CSI Data/CF_DNN/NH_MATRIX_GF_CDIT'

NH_MATRIX = np.load(outfile_NH_MATRIX+".npz")
NH_MATRIX.files
NH_MATRIX = NH_MATRIX['arr_0']

# Noisy_H ==> NH_MATRIX

#-----------------------------------------------------------------------------------------#
# useful for Rate, outage, Delta calculation ...  because it containing (Alpha, Pr, Ps) 


outfile_BF_NG_MATRIX = 'Imperfect CSI Data/CF_DNN/BF_NG_MATRIX_GF_CDIT'

dataset_test = np.load(outfile_BF_NG_MATRIX+".npz")
dataset_test.files
dataset_test = dataset_test['arr_0']

#-----------------------------------------------------------------------------------------#
# useful for Rate, outage, Delta calculation, because it containing G Matrix without noise 


outfile_BF_G_Benchmark = 'Imperfect CSI Data/CF_DNN/BF_G_Benchmark_GF_CDIT'

dataset = np.load(outfile_BF_G_Benchmark+".npz")
dataset.files
dataset = dataset['arr_0']




